In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt        
import torch
import torch.nn as nn       
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset     

In [4]:
df = pd.read_csv('D:\Dataset/IMDB Dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [6]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [7]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [8]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [9]:
df['sentiment'] = df["sentiment"].map({'positive': 1, 'negative': 0})

In [10]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [11]:
text = df['review'].values
labels = df['sentiment'].values
type(text)

numpy.ndarray

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features = 5000)
x = tfidf.fit_transform(text).toarray()
y = labels

In [13]:
print(x.shape)
print(y.shape)  
print(type(x))
print(type(y))

(50000, 5000)
(50000,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [14]:
split_ration = 0.8
split_index = int(split_ration * x.shape[0])

In [15]:
x_train, x_val = x[:split_index], x[split_index:]
y_train, y_val = y[:split_index], y[split_index:]

In [16]:
print("x_train:\n", x_train.shape)
print("y_train:\n", y_train.shape)
print("x_val:\n", x_val.shape)
print("y_val:\n", y_val.shape)

x_train:
 (40000, 5000)
y_train:
 (40000,)
x_val:
 (10000, 5000)
y_val:
 (10000,)


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset    

class SimpleNN(nn.Module):
    def __init__(self, input_dim, hiddem_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hiddem_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hiddem_dim, hiddem_dim)
        self.fc3 = nn.Linear(hiddem_dim, output_dim)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
        

In [18]:
input_dim = x_train.shape[1]
hidden_dim = 128
output_dim = 2

model = SimpleNN(input_dim, hidden_dim, output_dim)

In [19]:
model

SimpleNN(
  (fc1): Linear(in_features=5000, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
)

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [21]:
type(x_train)

numpy.ndarray

In [22]:
x_train_tensor = torch.from_numpy(x_train).float()
y_train_tensor = torch.from_numpy(y_train).long()
x_val_tensor = torch.from_numpy(x_val).float()
y_val_tensor = torch.from_numpy(y_val).long()

In [23]:
type(x_train_tensor)

torch.Tensor

In [24]:
train_dataset = torch

In [25]:
#objek dataset
train_dataset = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
val_Dataset = torch.utils.data.TensorDataset(x_val_tensor, y_val_tensor)

#objek dataloader
train_Loader = DataLoader(train_dataset, batch_size =64, shuffle = True)
val_Loader = DataLoader(val_Dataset, batch_size =64, shuffle = False  )


In [26]:
def train(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):

    #device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #print(f'Using device: {device}')

    #model.to(device)
    
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        model.eval()
        val_loss = 0
        correct =0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                val_loss = val_loss + criterion(outputs, labels).item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
        val_accuracy = 100 * correct / total
        print(f'Epoch: {epoch+1}/{num_epochs}')
        print(f'Training loss : {loss.item():.4f}')
        print(f'Validation Accuracy: {val_accuracy :.2f}%')
        print(f'Validation Loss : {val_loss/len(val_loader):4f}')

In [27]:
training_model = train(model = model, criterion=criterion, optimizer=optimizer, train_loader=train_Loader, val_loader=val_Loader, num_epochs=3)
training_model

Epoch: 1/3
Training loss : 0.3244
Validation Accuracy: 88.65%
Validation Loss : 0.280066
Epoch: 2/3
Training loss : 0.2889
Validation Accuracy: 88.64%
Validation Loss : 0.281798
Epoch: 3/3
Training loss : 0.2225
Validation Accuracy: 88.68%
Validation Loss : 0.289048


TEST REAL MODEL YANG SUDAH DI TRAINING DENGAN DATA SAMPLE YANG BARU

In [30]:
review_baru = ["I really love this movie, it was fantastic and thrilling!","the main character is inconsistent, has no stance and the story is all over the place", "The movie was very  and too long, I didn't like it at all."]

In [31]:
new_review_tfidf = tfidf.transform(review_baru).toarray()
new_review_tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [32]:
new_review_tensor = torch.from_numpy(new_review_tfidf).float()
new_review_tensor

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [33]:
with torch.no_grad():
    model.eval()
    outputs = model(new_review_tensor)
    _, predicted = torch.max(outputs, 1)
    predicted_labels = ['positive' if label == 1 else 'negative' for label in predicted.numpy()]

In [34]:
predicted_labels

['positive', 'negative', 'negative']